# 🚨 B.RED.DY (Kaggle Agents Intensive - Capstone Project)

_“Ensuring your Plan B not to turning red”_

**Author:** Oleg Smirnov

**LinkedIn:** [linkedin.com/in/osmirnov](https://www.linkedin.com/in/osmirnov)


## 🏆 Track: Concierge Agents

## 🎤 The Pitch
**Problem:** Planning complex events (dates, trips, concerts) is stressful and error-prone. Unexpected factors like weather changes, traffic jams, or sold-out tickets can ruin a perfect plan, and coordinating with others adds another layer of chaos.

**Solution:** The **B.RED.DY** (Defensive Planner) is a resilient Multi-Agent System that doesn't just plan—it *anticipates failure*. Using a "Defensive Loop" architecture, it proactively checks for risks (weather, inventory, traffic) and employs a Recovery Agent to fix issues or suggest alternatives. It even handles social coordination (A2A) to prevent fashion disasters.

**Value:** By automating the "Plan B" process, this agent transforms fragile itineraries into robust experiences. It saves users hours of frantic re-booking when disruptions occur and ensures social harmony through proactive coordination, ultimately delivering peace of mind for high-stakes events.

## 🔑 Key Features Implemented
This project demonstrates the following advanced agent concepts:
1.  **Multi-Agent System:** Uses `SequentialAgent`, `ParallelAgent` (MoE), and `LoopAgent` for robust orchestration.
2.  **Sessions & Memory:** Leverages ADK `Session` state for data consistency (canonical weather) and A2A coordination.
3.  **A2A Protocol:** Implements Agent-to-Agent coordination to resolve conflicts (e.g., outfit color clashes) between users. *(Note: This is a local simulation using shared session state; production A2A would use remote endpoints and Agent Cards.)*
4.  **Model Context Protocol (MCP):** While this notebook uses local mock tools for portability, a production version would implement external integrations (Weather, Transport, Commerce) as **MCP Servers**. This would provide standardized, secure access to real-world APIs like Uber, Ticketmaster, or WeatherAPI.
5.  **Tools:** Custom tools with "Chaos Engineering" (simulated failures) to demonstrate agent resilience.
6.  **Observability:** Comprehensive console logging and tracing of agent steps, tool calls, and state changes to monitor execution flow.

## Architecture

```
                    ┌─────────────────────────────────┐
                    │      User Goal & Context        │
                    └──────────────┬──────────────────┘
                                   │
                    ┌──────────────▼──────────────────┐
                    │   LoopAgent (defensive_loop)    │
                    │      max_iterations=2           │
                    └──┬───────────────────────────┬──┘
                       │                           │
          ┌────────────▼──────────┐    ┌───────────▼──────────┐
          │ SequentialAgent       │    │  recovery_agent      │
          │ (planning_workflow)   │    │  • 📰 Fetch news     │
          │                       │    │  • Fix tool failures │
          │ ┌──────────────────┐  │    │  • 👔 Outfit Coord   │
          │ │ weather_fetcher  │  │    │  • Or exit_loop()    │
          │ │ → weather_data   │  │    └───────────┬──────────┘
          │ ├──────────────────┤  │                │
          │ │ ParallelAgent    │  │    ┌───────────▼──────────┐
          │ │ (expert_team)    │  │    │  📰 get_event_news   │
          │ │ ┌─────────────┐  │  │    │  👔 get_coordinated_ │
          │ │ │🚗 Transport │  │ │    │      outfit (A2A)    │
          │ │ │☀️  Weather  │  │ │    └──────────────────────┘
          │ │ │🛒 Merchandis│  │ │  ← Experts run PARALLEL
          │ │ └─────────────┘  │ │     read {weather_data}
          │ ├──────────────────┤ │
          │ │ planner_agent    │ │  ← AGGREGATOR: combines
          │ │ (aggregator)     │ │     all expert outputs
          │ ├──────────────────┤ │
          │ │ risk_analyst     │ │
          │ └──────────────────┘ │
          └──────────────────────┘

State Flow (via output_key):
  weather_fetcher → weather_data (canonical weather for all agents)
  expert_team (ParallelAgent) runs CONCURRENTLY:
    → weather_expert → weather_analysis
    → transportation_expert → transportation_analysis  
    → merchandising_expert → merchandising_analysis
  planner → initial_plan (aggregates all expert outputs)
  risk_analyst → risk_assessment
  recovery → initial_plan (overwrites if failures found)
  
  Data Consistency:
    weather_data → shared by ALL experts via session.state
    Expert outputs → aggregated by planner_agent
  
  News Context (when failures occur):
    recovery_agent → get_event_news() → explains WHY plans failed

  A2A Coordination (when plan succeeds):
    recovery_agent → get_coordinated_outfit() → prevents fashion conflicts
```

In [ ]:
# Standard Kaggle boilerplate
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print("✅ Environment initialized")

## 📦 Installation & Setup

In [ ]:
%pip install -q google-adk

print("=" * 70)
print("✅ Installation Complete!")
print("=" * 70)
print("  ✓ google-adk (Google Agent Development Kit)")
print("=" * 70)

## 🔑 Configuration

In [ ]:
# Load API key
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ API key loaded from Kaggle Secrets")
except Exception as e:
    print(f"⚠️  Using environment variable: {e}")
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "YOUR_API_KEY_HERE")

# Import Google ADK
from google.genai import types
from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import ToolContext
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

print("🤖 Google ADK configured")

## 📊 Observability Layer

This notebook uses a **lightweight observability solution** (Python `logging` + custom `MetricsCollector`) suitable for demos and local development.

**Production deployment** would use enterprise observability:
- **Google Cloud Operations Suite**: Cloud Logging, Cloud Monitoring, Cloud Trace
- **OpenTelemetry**: Industry-standard distributed tracing and metrics
- **APM Platforms**: Datadog, New Relic, Dynatrace for full-stack monitoring
- **AI/LLM Tools**: LangSmith, Arize AI, MLflow for model observability

Demo implementation logs to console output and tracks tool execution metrics in-memory, which is sufficient for the Capstone but would need centralized logging, alerting, and dashboards for production use.


In [ ]:
import logging
import json
import time
from collections import defaultdict
from functools import wraps

class MetricsCollector:
    """
    Lightweight metrics collector for tracking tool execution statistics.
    """
    
    def __init__(self):
        self.metrics = {
            "tool_calls_total": 0,
            "tool_calls_success": 0,
            "tool_calls_failure": 0,
            "tool_execution_times": defaultdict(list),
            "session_starts": 0,
            "session_completions": 0,
            "session_failures": 0
        }
    
    def record_tool_call(self, tool_name: str, success: bool, execution_time: float):
        """Record a tool execution."""
        self.metrics["tool_calls_total"] += 1
        if success:
            self.metrics["tool_calls_success"] += 1
        else:
            self.metrics["tool_calls_failure"] += 1
        
        self.metrics["tool_execution_times"][tool_name].append(execution_time)
    
    def record_session_start(self):
        """Record session start."""
        self.metrics["session_starts"] += 1
    
    def record_session_completion(self, success: bool):
        """Record session completion."""
        if success:
            self.metrics["session_completions"] += 1
        else:
            self.metrics["session_failures"] += 1
    
    def report(self) -> str:
        """Generate a metrics report."""
        report_lines = [
            "📊 OBSERVABILITY METRICS REPORT",
            "=" * 50,
            f"Total Tool Calls: {self.metrics['tool_calls_total']}",
            f"Successful Tool Calls: {self.metrics['tool_calls_success']}",
            f"Failed Tool Calls: {self.metrics['tool_calls_failure']}",
            f"Sessions Started: {self.metrics['session_starts']}",
            f"Sessions Completed: {self.metrics['session_completions']}",
            f"Sessions Failed: {self.metrics['session_failures']}",
            "",
            "Tool Execution Times (avg ms):"
        ]
        
        for tool_name, times in self.metrics["tool_execution_times"].items():
            if times:
                avg_time = sum(times) / len(times) * 1000  # Convert to ms
                min_time = min(times) * 1000
                max_time = max(times) * 1000
                report_lines.append(f"  {tool_name}: avg={avg_time:.1f}ms, min={min_time:.1f}ms, max={max_time:.1f}ms")
        
        if self.metrics["tool_calls_total"] > 0:
            success_rate = (self.metrics["tool_calls_success"] / self.metrics["tool_calls_total"]) * 100
            report_lines.append(f"")
            report_lines.append(f"Tool Success Rate: {success_rate:.1f}%")
        
        if self.metrics["session_starts"] > 0:
            completion_rate = (self.metrics["session_completions"] / self.metrics["session_starts"]) * 100
            report_lines.append(f"Session Completion Rate: {completion_rate:.1f}%")
        
        return "\n".join(report_lines)

# Global metrics collector instance
metrics_collector = MetricsCollector()

# Configure logging to write to both file and stdout
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        # logging.FileHandler('agent_execution.log', mode='w'),  # Overwrite on each run
        logging.StreamHandler()  # Also print to stdout
    ]
)

logger = logging.getLogger(__name__)

def trace_tool(func):
    """
    Decorator that traces tool execution, logging inputs/outputs, timing, and metrics.
    
    Args:
        func: The tool function to trace
    
    Returns:
        Wrapped function that logs execution details and updates metrics
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        tool_name = func.__name__
        start_time = time.time()
        
        # Log tool call start with inputs
        input_data = {
            "args": [str(arg) for arg in args],
            "kwargs": {k: str(v) for k, v in kwargs.items()}
        }
        logger.info(f"🔧 TOOL START: {tool_name} | INPUT: {json.dumps(input_data, indent=None)}")
        
        try:
            # Execute the tool
            result = func(*args, **kwargs)
            execution_time = time.time() - start_time
            
            # Determine success based on result structure
            # Most tools return JSON strings with "status" field
            success = True
            if isinstance(result, str):
                try:
                    parsed = json.loads(result)
                    if isinstance(parsed, dict) and parsed.get("status") == "failure":
                        success = False
                except json.JSONDecodeError:
                    pass  # Not JSON, assume success
            elif hasattr(result, 'status') and result.status == "failure":
                success = False
            
            # Record metrics
            metrics_collector.record_tool_call(tool_name, success, execution_time)
            
            # Log completion with output
            status_emoji = "✅" if success else "❌"
            logger.info(f"{status_emoji} TOOL COMPLETE: {tool_name} | SUCCESS: {success} | TIME: {execution_time:.3f}s | OUTPUT: {str(result)[:200]}{'...' if len(str(result)) > 200 else ''}")
            
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            metrics_collector.record_tool_call(tool_name, False, execution_time)
            
            logger.error(f"💥 TOOL ERROR: {tool_name} | TIME: {execution_time:.3f}s | ERROR: {str(e)}")
            raise
    
    return wrapper



## 🧰 Mock Tools with Chaos Engineering

In [ ]:
import json
import random
from enum import Enum

class PlanStatus(Enum):
    PLANNING = "PLANNING"
    ASSESSING = "ASSESSING"
    SIMULATING = "SIMULATING"
    RECOVERING = "RECOVERING"
    SUCCESS = "SUCCESS"
    FAILED = "FAILED"

# ═══════════════════════════════════════════════════════════════════════════════
# WEATHER TOOLS - Single Source of Truth via ADK Session State
# ═══════════════════════════════════════════════════════════════════════════════
#
# MCP NOTE: In a production environment, this would be implemented as an 
# MCP Server (Model Context Protocol) wrapping a real provider like 
# OpenWeatherMap or WeatherAPI.
#
# DATA CONSISTENCY STRATEGY:
# 1. get_weather_forecast() is the weather source (generates once)
# 2. Weather fetcher agent stores result via output_key="weather_data"
# 3. Other agents read {weather_data} from session and pass to their tools
# 4. Tools like analyze_weather_impact() and get_outfit_recommendation() 
#    receive weather as PARAMETERS (not random generation)
#
# This ensures ALL agents work with the SAME weather data!
# ═══════════════════════════════════════════════════════════════════════════════

@trace_tool
def get_weather_forecast(location: str, timeframe: str = "now", check_severe: bool = False) -> str:
    """
    weather source - generates weather data stored in session via output_key.
    
    This is the ONLY tool that generates random weather. All other weather-dependent
    tools should receive weather data as parameters from the session state {weather_data}.
    
    Args:
        location: Location to check weather for
        timeframe: "now" for current weather, or "6h", "12h", "24h", "48h" for forecast
        check_severe: If True, includes severe weather alerts and warnings
    
    Returns:
        JSON with weather data including TOP-LEVEL fields for easy session access:
        - condition, temperature, precipitation (for other agents to extract)
    """
    # 10% failure rate (chaos engineering)
    if random.random() < 0.10:
        return json.dumps({
            "status": "failure", 
            "condition": "Severe Storm", 
            "error": "Weather API unavailable",
            "location": location
        })
    
    # Map conditions to consistent precipitation values
    conditions_map = {
        "Clear": "None", "Sunny": "None", "Partly Cloudy": "None", "Cloudy": "None",
        "Light Rain": "Light Rain", "Rain": "Rain", "Heavy Rain": "Heavy Rain",
        "Snow": "Snow", "Storm": "Heavy Rain"
    }
    
    # Generate weather ONCE - becomes the session's weather
    condition = random.choice(list(conditions_map.keys()))
    temperature = random.randint(50, 95)
    humidity = random.randint(30, 95)
    wind_speed = random.randint(5, 35)
    precipitation = conditions_map[condition]
    
    # Quick check mode
    if timeframe == "now":
        result = {
            "status": "success",
            "location": location,
            "timeframe": "now",
            "current": {
                "condition": condition,
                "temperature": temperature,
                "humidity": humidity,
                "wind_speed": wind_speed
            },
            # TOP-LEVEL FIELDS for easy session access by other agents
            "condition": condition,
            "temperature": temperature,
            "precipitation": precipitation
        }
    else:
        # Detailed forecast mode
        hours = {"6h": 6, "12h": 12, "24h": 24, "48h": 48}.get(timeframe, 24)
        forecast_periods = []
        
        for i in range(0, hours, 6):
            period = {
                "time": f"+{i}h to +{i+6}h",
                "condition": condition if i == 0 else random.choice(list(conditions_map.keys())),
                "temperature": temperature + random.randint(-10, 10),
                "precipitation_chance": random.randint(0, 100),
                "wind_speed": wind_speed + random.randint(-5, 5),
                "humidity": humidity + random.randint(-10, 10)
            }
            forecast_periods.append(period)
        
        result = {
            "status": "success",
            "location": location,
            "timeframe": timeframe,
            "forecast": forecast_periods,
            "alerts": random.choice([[], ["Heat Advisory"], ["Storm Watch"], ["Wind Advisory"]]),
            # TOP-LEVEL FIELDS for easy session access
            "condition": condition,
            "temperature": temperature,
            "precipitation": precipitation
        }
    
    # Severe weather check
    if check_severe:
        has_severe = random.random() < 0.15
        if has_severe:
            severe_types = ["Thunderstorm", "Heavy Snow", "Ice Storm", "Tornado Watch", "Hurricane Watch"]
            result["severe_weather"] = {
                "detected": True,
                "type": random.choice(severe_types),
                "impact_level": random.choice(["moderate", "high", "severe"]),
                "advisory": "Consider rescheduling or taking precautions"
            }
        else:
            result["severe_weather"] = {
                "detected": False,
                "conditions": "Normal weather conditions expected"
            }
    
    return json.dumps(result)


# ═══════════════════════════════════════════════════════════════════════════════
# TRANSPORTATION TOOLS
# ═══════════════════════════════════════════════════════════════════════════════
#
# MCP NOTE: In production, these would be MCP Servers connecting to real-world
# transportation APIs (e.g., Uber/Lyft for taxis, Google Maps for traffic,
# City Transit APIs for public transport).
# ═══════════════════════════════════════════════════════════════════════════════

@trace_tool
def book_taxi(destination: str, pickup_time: str, pickup_location: str = "current", check_only: bool = False) -> str:
    """
    Unified taxi tool.
    
    Args:
        destination: Where to go
        pickup_time: When to be picked up
        pickup_location: Where to be picked up (default: current location)
        check_only: If True, only checks availability without booking
    
    Returns:
        JSON with availability info and/or booking confirmation
    """
    available = random.randint(0, 20)
    surge = round(random.uniform(1.0, 4.5), 1) if available < 5 else 1.0
    estimated_wait = f"{random.randint(2, 15)} min" if available > 0 else "30+ min"
    
    # Check-only mode (replaces check_taxi_availability)
    if check_only:
        result = {
            "status": "success",
            "mode": "availability_check",
            "location": pickup_location,
            "available_taxis": available,
            "surge_multiplier": surge,
            "estimated_wait": estimated_wait,
            "recommendation": "Book now" if available > 5 else "Consider alternatives"
        }
        return json.dumps(result)
    
    # Booking mode with 20% failure rate
    if random.random() < 0.20 or available == 0:
        result = {
            "status": "failure", 
            "error": "No cars available",
            "available_taxis": available,
            "surge_multiplier": surge,
            "recommendation": "Try again in a few minutes or use transit"
        }
    else:
        result = {
            "status": "success",
            "mode": "booked",
            "booking_id": f"TAXI{random.randint(1000, 9999)}",
            "destination": destination,
            "pickup_location": pickup_location,
            "pickup_time": pickup_time,
            "eta": estimated_wait,
            "surge_multiplier": surge,
            "estimated_fare": f"${random.randint(15, 60) * surge:.2f}"
        }
    return json.dumps(result)

@trace_tool
def get_transit_info(route: str, departure_time: str = "now") -> str:
    """
    Unified transit tool.
    
    Args:
        route: Transit route to check
        departure_time: When planning to depart
    
    Returns:
        JSON with platform info, delays, crowding, and service alerts
    """
    # 8% failure rate (from original check_transit)
    if random.random() < 0.08:
        return json.dumps({
            "status": "failure", 
            "error": "Service disruption",
            "route": route,
            "recommendation": "Consider alternative transportation"
        })
    
    delay_mins = random.choice([0, 0, 0, 5, 10, 15, 25])  # Most times no delay
    
    result = {
        "status": "success",
        "route": route,
        "departure_time": departure_time,
        "platform": random.choice(["A", "B", "C", "1", "2", "3"]),
        "delay_minutes": delay_mins,
        "service_status": "Normal" if delay_mins == 0 else "Delayed",
        "next_arrival": f"{random.randint(3, 12)} min",
        "crowding_level": random.choice(["Low", "Medium", "High"]),
        "alerts": [] if delay_mins == 0 else [f"Signal issues causing {delay_mins}min delays"]
    }
    return json.dumps(result)

@trace_tool
def get_real_time_traffic(origin: str, destination: str, time: str = "now") -> str:
    """Get real-time traffic conditions with route alternatives."""
    conditions = ["Light", "Moderate", "Heavy", "Severe"]
    traffic_level = random.choice(conditions)
    
    result = {
        "status": "success",
        "origin": origin,
        "destination": destination,
        "traffic_level": traffic_level,
        "estimated_duration": random.randint(15, 60),
        "alternative_routes": [
            {"route": "Highway", "duration": random.randint(20, 45), "traffic": random.choice(conditions)},
            {"route": "Surface Streets", "duration": random.randint(25, 50), "traffic": random.choice(conditions)}
        ],
        "incidents": random.choice([None, "Accident on Main St", "Road work on 5th Ave"])
    }
    return json.dumps(result)

# ═══════════════════════════════════════════════════════════════════════════════
# MERCHANDISING TOOLS
# ═══════════════════════════════════════════════════════════════════════════════
#
# MCP NOTE: In production, these would be MCP Servers integrating with e-commerce
# platforms (Amazon, Shopify), ticket vendors (Ticketmaster), or inventory systems.
# ═══════════════════════════════════════════════════════════════════════════════

@trace_tool
def search_products(query: str, category: str = "all", check_specific_item: str = None) -> str:
    """
    Unified product search tool.
    
    Args:
        query: Search query for products
        category: Product category to filter by
        check_specific_item: If provided, checks availability of this specific item (replaces check_inventory)
    
    Returns:
        JSON with product results including availability, or specific item availability
    """
    # Specific item check mode (replaces check_inventory) - 15% failure rate
    if check_specific_item:
        if random.random() < 0.15:
            return json.dumps({
                "status": "failure",
                "error": "Out of stock",
                "item": check_specific_item,
                "alternatives": [f"Similar to {check_specific_item} - Option A", f"Similar to {check_specific_item} - Option B"]
            })
        else:
            return json.dumps({
                "status": "success",
                "item": check_specific_item,
                "available": True,
                "quantity_available": random.randint(1, 50),
                "location": random.choice(["Aisle 3", "Section B", "Online Only", "In Store"])
            })
    
    # General search mode
    num_results = random.randint(3, 8)
    products = []
    
    for i in range(num_results):
        availability = random.choice(["In Stock", "Limited Stock", "Pre-order", "Out of Stock"])
        product = {
            "id": f"PROD{random.randint(1000, 9999)}",
            "name": f"{query} - Variant {i+1}",
            "price": random.randint(20, 500),
            "rating": round(random.uniform(3.5, 5.0), 1),
            "availability": availability,
            "in_stock": availability != "Out of Stock",
            "quantity_available": random.randint(0, 50) if availability != "Out of Stock" else 0,
            "shipping": random.choice(["Same Day", "2-3 Days", "1 Week"])
        }
        products.append(product)
    
    result = {
        "status": "success",
        "query": query,
        "category": category,
        "results_count": num_results,
        "products": products,
        "in_stock_count": sum(1 for p in products if p["in_stock"])
    }
    return json.dumps(result)

@trace_tool
def find_best_deal(item: str, vendors: list = None, include_promotions: bool = True) -> str:
    """
    Unified deal-finding tool.
    
    Args:
        item: Item to find deals for
        vendors: List of vendors to compare (default: common vendors)
        include_promotions: If True, includes available promo codes and discounts
    
    Returns:
        JSON with price comparisons, best deal, and applicable promotions
    """
    if vendors is None:
        vendors = ["Store A", "Store B", "Store C", "Online Mart"]
    
    comparisons = []
    for vendor in vendors:
        available = random.random() > 0.2  # 80% availability
        if available:
            comparison = {
                "vendor": vendor,
                "price": random.randint(30, 300),
                "in_stock": True,
                "delivery": random.choice(["Pickup", "1-2 Days", "3-5 Days"]),
                "rating": round(random.uniform(3.0, 5.0), 1)
            }
        else:
            comparison = {
                "vendor": vendor,
                "in_stock": False,
                "estimated_restock": f"{random.randint(3, 14)} days"
            }
        comparisons.append(comparison)
    
    # Find best deal
    available_deals = [c for c in comparisons if c.get("in_stock", False)]
    best_deal = min(available_deals, key=lambda x: x["price"]) if available_deals else None
    
    result = {
        "status": "success",
        "item": item,
        "comparisons": comparisons,
        "best_deal": best_deal,
        "total_vendors": len(vendors)
    }
    
    # Include promotions (replaces check_promotions)
    if include_promotions:
        has_promo = random.random() > 0.4  # 60% chance of promotions
        
        if has_promo:
            promotions = []
            num_promos = random.randint(1, 4)
            
            promo_types = [
                {"type": "Percentage Off", "value": f"{random.randint(10, 50)}% off"},
                {"type": "Buy One Get One", "value": "BOGO 50% off"},
                {"type": "Fixed Discount", "value": f"${random.randint(10, 100)} off"},
                {"type": "Bundle Deal", "value": "Save 30% on bundle"}
            ]
            
            for i in range(num_promos):
                promo = random.choice(promo_types).copy()
                promo["expires"] = f"{random.randint(1, 7)} days"
                promo["code"] = f"PROMO{random.randint(100, 999)}"
                promotions.append(promo)
            
            result["promotions"] = {
                "available": True,
                "codes": promotions,
                "recommendation": "Apply best promo code at checkout"
            }
            
            # Calculate best deal with promo
            if best_deal and promotions:
                best_promo = promotions[0]
                result["best_deal_with_promo"] = {
                    "vendor": best_deal["vendor"],
                    "original_price": best_deal["price"],
                    "promo_code": best_promo["code"],
                    "promo_value": best_promo["value"]
                }
        else:
            result["promotions"] = {
                "available": False,
                "message": "No active promotions currently"
            }
    
    return json.dumps(result)

@trace_tool
def get_purchase_recommendations(context: str, budget: int = None) -> str:
    """Get personalized purchase recommendations based on context."""
    recommendations = []
    num_recs = random.randint(3, 6)
    
    for i in range(num_recs):
        rec = {
            "item": f"Recommended Item {i+1}",
            "reason": random.choice([
                "Best seller",
                "Highly rated",
                "Perfect for occasion",
                "Trending now",
                "Great value"
            ]),
            "price": random.randint(25, 250),
            "rating": round(random.uniform(4.0, 5.0), 1),
            "match_score": random.randint(75, 100)
        }
        
        if budget:
            if rec["price"] <= budget:
                recommendations.append(rec)
        else:
            recommendations.append(rec)
    
    result = {
        "status": "success",
        "context": context,
        "budget": budget,
        "recommendations": recommendations,
        "total_options": len(recommendations)
    }
    return json.dumps(result)

@trace_tool
def purchase_tickets(event: str, time: str, quantity: int = 1) -> str:
    """Purchase tickets with 30% failure rate."""
    if random.random() < 0.30:
        result = {"status": "failure", "error": "Sold out", "alternatives": ["6 PM", "9 PM"]}
    else:
        result = {"status": "success", "ticket_id": f"TKT{random.randint(10000, 99999)}",
                  "event": event, "price": random.randint(50, 200)}
    return json.dumps(result)

@trace_tool
def analyze_weather_impact(route: str, mode: str, weather_condition: str) -> str:
    """
    Analyze how weather impacts transportation mode using PASSED weather data.
    
    IMPORTANT: This tool receives weather from session state via agent instruction.
    The agent reads {weather_data}.condition and passes it to this tool.
    This ensures weather impact analysis is CONSISTENT with the weather forecast.
    
    Args:
        route: The route being analyzed
        mode: Transportation mode ("taxi", "transit", "walk", "drive")
        weather_condition: Weather condition from session (e.g., "Rain", "Snow", "Clear")
    
    Returns:
        JSON with impact analysis based on actual session weather
    """
    # Normalize condition to match impact matrix keys
    condition_normalized = weather_condition
    if weather_condition in ["Sunny", "Partly Cloudy", "Cloudy"]:
        condition_normalized = "Clear"
    elif weather_condition in ["Light Rain", "Heavy Rain"]:
        condition_normalized = "Rain"
    
    impact_matrix = {
        "taxi": {"Clear": "none", "Rain": "minor", "Snow": "moderate", "Storm": "severe"},
        "transit": {"Clear": "none", "Rain": "minor", "Snow": "major", "Storm": "severe"},
        "walk": {"Clear": "none", "Rain": "moderate", "Snow": "severe", "Storm": "unsafe"},
        "drive": {"Clear": "none", "Rain": "minor", "Snow": "moderate", "Storm": "severe"}
    }
    
    impact = impact_matrix.get(mode, {}).get(condition_normalized, "unknown")
    
    result = {
        "status": "success",
        "data_source": "session_weather",  # Indicates data consistency!
        "weather_condition": weather_condition,
        "weather_normalized": condition_normalized,
        "mode": mode,
        "route": route,
        "impact": impact,
        "delay_risk": "high" if impact in ["severe", "major", "unsafe"] else "low",
        "delay_estimate": f"{random.randint(5, 30)} min" if impact in ["severe", "major"] else "0 min",
        "recommendation": "Use alternative transport" if impact in ["severe", "unsafe"] else "Proceed as planned"
    }
    return json.dumps(result)

@trace_tool
def get_outfit_recommendation(activity: str, temperature: int, condition: str, precipitation: str = "None") -> str:
    """
    Recommend outfit based on PASSED weather data (not random generation).
    
    IMPORTANT: This tool receives weather from session state via agent instruction.
    The agent reads {weather_data} and passes temp/condition/precipitation to this tool.
    This ensures outfit recommendations are CONSISTENT with the weather forecast.
    
    Args:
        activity: What activity the outfit is for
        temperature: Temperature in Fahrenheit (from session weather_data)
        condition: Weather condition string (from session weather_data)
        precipitation: Precipitation type (from session weather_data)
    
    Returns:
        JSON with outfit recommendation based on actual session weather
    """
    # Derive outfit type from ACTUAL temperature (not random!)
    if temperature >= 85:
        outfit_type = "Hot"
    elif temperature >= 70:
        outfit_type = "Warm"
    elif temperature >= 55:
        outfit_type = "Mild"
    elif temperature >= 40:
        outfit_type = "Cool"
    else:
        outfit_type = "Cold"
    
    outfit_map = {
        "Hot": {"base": "Light clothing", "top": "T-shirt", "bottom": "Shorts", "accessories": ["Sunglasses", "Hat"]},
        "Warm": {"base": "Summer wear", "top": "Short sleeves", "bottom": "Light pants", "accessories": ["Sunglasses"]},
        "Mild": {"base": "Light layers", "top": "Long sleeves", "bottom": "Jeans", "accessories": ["Light jacket"]},
        "Cool": {"base": "Layers", "top": "Sweater", "bottom": "Pants", "accessories": ["Jacket"]},
        "Cold": {"base": "Warm layers", "top": "Heavy coat", "bottom": "Warm pants", "accessories": ["Scarf", "Gloves", "Hat"]}
    }
    
    recommendation = outfit_map[outfit_type].copy()
    recommendation["accessories"] = recommendation["accessories"].copy()
    
    # Add rain/snow gear based on ACTUAL precipitation from session
    if precipitation in ["Light Rain", "Rain", "Heavy Rain"]:
        recommendation["accessories"].extend(["Umbrella", "Waterproof jacket"])
    elif precipitation == "Snow":
        recommendation["accessories"].extend(["Winter boots", "Warm coat"])
    
    result = {
        "status": "success",
        "data_source": "session_weather",  # Indicates data consistency!
        "activity": activity,
        "weather_used": {
            "temperature": temperature,
            "condition": condition,
            "precipitation": precipitation
        },
        "outfit_category": outfit_type,
        "recommendation": recommendation
    }
    return json.dumps(result)


# ═══════════════════════════════════════════════════════════════════════════════
# A2A COORDINATED OUTFIT TOOL - Session State Based Color & Style Coordination
# ═══════════════════════════════════════════════════════════════════════════════
#
# DISCLAIMER:
# This is a DEMONSTRATION implementation of Agent-to-Agent (A2A) coordination.
# A production-grade A2A system would typically involve:
# - Remote agent endpoints (REST/gRPC)
# - Standardized Agent Cards for capability discovery
# - Secure handshake protocols
# - Distributed state management
#
# For this notebook, we simulate A2A behavior using shared Session State to
# demonstrate the *concept* of multi-user coordination without requiring
# complex network infrastructure.
#
# - Uses ToolContext to access session state
# - Stores outfit assignments with event-scoped keys
# - Enables cross-agent coordination via shared session state
# ═══════════════════════════════════════════════════════════════════════════════

@trace_tool
def get_coordinated_outfit(
    tool_context: ToolContext,
    event_id: str,
    suggested_outfit: str,
    preferred_color: str = None,
    preferred_style: str = None
) -> str:
    """
    Coordinate outfit colors and styles between users at the same event.
    
    Takes a suggested outfit item from get_outfit_recommendation (e.g. "Sunglasses", 
    "Hat", "Light jacket") and coordinates colors/styles so users DON'T match.
    
    This tool does NOT determine what to wear based on weather - that's handled
    by get_outfit_recommendation. This tool only handles A2A coordination.
    
    Uses session's user_id for reliable user identification instead of user-provided names.
    
    Implements ADK A2A pattern locally using session state instead of remote agents.
    
    Args:
        tool_context: ADK ToolContext providing access to session state and user_id
        event_id: Unique identifier for the event (used as session state key)
        suggested_outfit: Outfit item from get_outfit_recommendation (e.g. "Hat", "Light jacket")
        preferred_color: User's preferred color (may be changed if conflict)
        preferred_style: User's preferred style (may be changed if conflict)
    
    Returns:
        JSON with coordinated outfit colors/styles and conflict resolution info
    """
    # Get reliable user identifier from session context (not user-provided name)
    user_id = tool_context.user_id
    
    # State key for this event's outfit assignments (following ADK session patterns)
    event_key = f"event:{event_id}:outfits"
    
    # Get existing outfit assignments for this event from session state
    existing_outfits = tool_context.state.get(event_key, {})
    
    # Build lists of already-used colors and styles at this event
    used_colors = []
    used_styles = []
    for uid, outfit_info in existing_outfits.items():
        if uid != user_id:
            if outfit_info.get("color"):
                used_colors.append(outfit_info["color"].lower())
            if outfit_info.get("style"):
                used_styles.append(outfit_info["style"].lower())
    
    # Available color palettes for coordination
    color_options = [
        "navy blue", "emerald green", "burgundy", "charcoal gray", 
        "royal purple", "forest green", "midnight blue", "silver gray",
        "rust orange", "teal", "coral", "slate blue", "olive green",
        "dusty rose", "classic black", "ivory white"
    ]
    
    # Style variations for common outfit items
    style_options = {
        "hat": ["fedora", "baseball cap", "beanie", "bucket hat", "sun hat", "newsboy cap"],
        "sunglasses": ["aviator", "wayfarer", "round", "cat-eye", "sport", "oversized"],
        "light jacket": ["denim", "bomber", "windbreaker", "cardigan", "blazer", "utility"],
        "jacket": ["leather", "puffer", "parka", "trench", "peacoat", "field jacket"],
        "scarf": ["silk", "wool", "cashmere", "infinity", "blanket", "bandana"],
        "default": ["classic", "modern", "vintage", "sporty", "casual", "elegant"]
    }
    
    # Resolve color conflicts
    color_conflict = False
    conflicting_color_user = None
    original_color = preferred_color
    assigned_color = preferred_color
    
    if preferred_color and preferred_color.lower() in used_colors:
        color_conflict = True
        for uid, outfit_info in existing_outfits.items():
            if outfit_info.get("color", "").lower() == preferred_color.lower():
                conflicting_color_user = uid
                break
        # Find alternative color
        for alt_color in color_options:
            if alt_color.lower() not in used_colors:
                assigned_color = alt_color
                break
        else:
            assigned_color = "classic black"
    elif not preferred_color:
        # Auto-assign a color that doesn't conflict
        for alt_color in color_options:
            if alt_color.lower() not in used_colors:
                assigned_color = alt_color
                break
        else:
            assigned_color = "classic black"
    
    # Resolve style conflicts
    style_conflict = False
    conflicting_style_user = None
    original_style = preferred_style
    assigned_style = preferred_style
    
    # Get style options for this outfit item
    outfit_lower = suggested_outfit.lower()
    available_styles = style_options.get(outfit_lower, style_options["default"])
    
    if preferred_style and preferred_style.lower() in used_styles:
        style_conflict = True
        for uid, outfit_info in existing_outfits.items():
            if outfit_info.get("style", "").lower() == preferred_style.lower():
                conflicting_style_user = uid
                break
        # Find alternative style
        for alt_style in available_styles:
            if alt_style.lower() not in used_styles:
                assigned_style = alt_style
                break
        else:
            assigned_style = available_styles[0] if available_styles else "classic"
    elif not preferred_style:
        # Auto-assign a style that doesn't conflict
        for alt_style in available_styles:
            if alt_style.lower() not in used_styles:
                assigned_style = alt_style
                break
        else:
            assigned_style = available_styles[0] if available_styles else "classic"
    
    # Save this user's outfit choice to session state for future A2A coordination
    existing_outfits[user_id] = {
        "outfit_item": suggested_outfit,
        "color": assigned_color,
        "style": assigned_style,
        "event": event_id,
        "color_conflict_with": conflicting_color_user if color_conflict else None,
        "style_conflict_with": conflicting_style_user if style_conflict else None
    }
    tool_context.state[event_key] = existing_outfits
    
    conflict_detected = color_conflict or style_conflict
    
    result = {
        "status": "conflict_resolved" if conflict_detected else "success",
        "a2a_coordination": True,
        "user_id": user_id,
        "event": event_id,
        "outfit_item": suggested_outfit,
        "coordinated_outfit": {
            "item": suggested_outfit,
            "color": assigned_color,
            "style": assigned_style,
            "description": f"{assigned_color} {assigned_style} {suggested_outfit}".strip()
        },
        "color_coordination": {
            "requested_color": original_color,
            "assigned_color": assigned_color,
            "conflict_detected": color_conflict,
            "conflicting_user": conflicting_color_user,
            "colors_already_taken": used_colors,
            "resolution": f"Changed from {original_color} to {assigned_color}" if color_conflict else "No conflict"
        },
        "style_coordination": {
            "requested_style": original_style,
            "assigned_style": assigned_style,
            "conflict_detected": style_conflict,
            "conflicting_user": conflicting_style_user,
            "styles_already_taken": used_styles,
            "resolution": f"Changed from {original_style} to {assigned_style}" if style_conflict else "No conflict"
        },
        "other_attendees": {
            uid: {"color": info.get("color"), "style": info.get("style")} 
            for uid, info in existing_outfits.items() if uid != user_id
        }
    }
    
    return json.dumps(result)


# ═══════════════════════════════════════════════════════════════════════════════
# NEWS SIMULATION TOOL - Contextual News Based on Events
# ═══════════════════════════════════════════════════════════════════════════════
#
# MCP NOTE: In production, this would be an MCP Server connecting to a news
# aggregator API (e.g., NewsAPI, Bing News) to fetch real-time context.
#
# This tool provides simulated news that helps users understand WHY their plans
# might be affected. When weather is severe, transportation fails, or events
# are impacted, this tool generates relevant news headlines and articles.
#
# NO CHAOS ENGINEERING - This is purely informational news simulation.
# ═══════════════════════════════════════════════════════════════════════════════

@trace_tool
def get_event_news(location: str, category: str = "general", weather_condition: str = None, context: str = None) -> str:
    """
    Get simulated news related to the user's event and current conditions.
    
    This tool helps users understand why certain plans might not work by
    providing contextual news that explains the current situation in the area.
    
    NO CHAOS ENGINEERING - This is purely informational, always returns success.
    
    Args:
        location: The location to get news for (e.g., "Chicago", "downtown", "Central Park")
        category: News category - "weather", "transportation", "entertainment", "general"
        weather_condition: Current weather condition from session (e.g., "Storm", "Snow", "Rain")
        context: Additional context about the situation (e.g., "flight delays", "sold out concert")
    
    Returns:
        JSON with relevant news articles based on the situation
    """
    import datetime
    
    news_articles = []
    
    # Weather-related news
    weather_headlines = {
        "Storm": [
            f"⛈️ BREAKING: Huge Storm Hits {location} Area - Residents Urged to Stay Indoors",
            f"🌪️ Severe Weather Alert: {location} Under Storm Warning Until Tonight",
            f"⚡ Storm Causes Widespread Power Outages Across {location} Region"
        ],
        "Severe Storm": [
            f"🚨 EMERGENCY: Severe Storm Batters {location} - All Non-Essential Travel Suspended",
            f"⛈️ {location} Declares Weather Emergency as Storm Intensifies",
            f"📺 Stay Home and Watch TV: {location} Storm Makes Travel Dangerous"
        ],
        "Heavy Rain": [
            f"🌧️ Heavy Rain Causes Flash Flooding in {location} - Roads Closed",
            f"☔ {location} Residents Advised to Avoid Low-Lying Areas Due to Heavy Rain",
            f"🚗 Traffic Snarled Across {location} as Heavy Rain Reduces Visibility"
        ],
        "Snow": [
            f"❄️ Snow Blankets {location} - Schools and Offices Closed",
            f"🌨️ Winter Storm Dumps Several Inches on {location} Area",
            f"⚠️ {location} Snow Emergency: Only Essential Travel Recommended"
        ],
        "Ice Storm": [
            f"🧊 Ice Storm Creates Treacherous Conditions in {location}",
            f"⚠️ {location} Roads Like 'Skating Rinks' - Stay Off Roads If Possible",
            f"🚨 Ice Storm Causes Multiple Accidents Across {location} Metro"
        ],
        "Tornado Watch": [
            f"🌪️ ALERT: Tornado Watch Issued for {location} Area",
            f"⚠️ {location} Residents Should Seek Shelter - Tornado Conditions Possible",
            f"📢 Take Cover: Tornado Spotted Near {location}"
        ],
        "Hurricane Watch": [
            f"🌀 Hurricane Approaches {location} - Evacuations Underway",
            f"🚨 {location} Braces for Hurricane Impact - Board Up Windows Now",
            f"⛽ Gas Stations Running Dry as {location} Prepares for Hurricane"
        ],
        "Thunderstorm": [
            f"⛈️ Thunderstorms Roll Through {location} - Lightning Strikes Reported",
            f"🌩️ Outdoor Events Cancelled in {location} Due to Thunderstorm Activity",
            f"⚡ {location} Residents Advised to Unplug Electronics During Storms"
        ],
        "Heat Advisory": [
            f"🌡️ Extreme Heat Warning: {location} Temperatures to Exceed 100°F",
            f"☀️ {location} Opens Cooling Centers as Heat Wave Continues",
            f"🥵 Stay Hydrated: {location} Under Dangerous Heat Advisory"
        ]
    }
    
    # Transportation-related news
    transport_headlines = {
        "flight_delays": [
            f"✈️ Major Flight Delays at {location} Airport - Check Your Flight Status",
            f"🛫 Airlines Cancel Dozens of Flights at {location} Due to Weather",
            f"⏰ Passengers Stranded as {location} Airport Experiences Massive Delays"
        ],
        "traffic": [
            f"🚗 Major Traffic Backup on {location} Highway - Expect 2+ Hour Delays",
            f"🚧 Multi-Vehicle Accident Causes Gridlock in {location}",
            f"⚠️ Avoid {location} Downtown Area - Heavy Congestion Reported"
        ],
        "transit_disruption": [
            f"🚇 {location} Transit System Reports Major Service Disruptions",
            f"🚌 Buses Running on Limited Schedule Due to {location} Conditions",
            f"🚃 Train Service Suspended in Parts of {location}"
        ],
        "taxi_shortage": [
            f"🚕 Surge Pricing Hits Record High as {location} Faces Taxi Shortage",
            f"📱 Rideshare Wait Times Exceed 30 Minutes in {location}",
            f"🚖 {location} Transportation Crisis: Cabs and Rideshares in High Demand"
        ]
    }
    
    # Entertainment-related news
    entertainment_headlines = {
        "sold_out": [
            f"🎭 Hot Ticket: {location} Event Sells Out in Record Time",
            f"🎫 Fans Disappointed as {location} Show Reaches Capacity",
            f"🎤 Scalpers Asking 3x Face Value for Sold-Out {location} Event"
        ],
        "cancelled": [
            f"❌ Popular {location} Event Cancelled Due to Unforeseen Circumstances",
            f"😢 {location} Show Postponed - Refunds Available at Point of Purchase",
            f"🎭 Breaking: Tonight's {location} Performance Cancelled"
        ],
        "venue_closed": [
            f"🏛️ {location} Venue Temporarily Closed for Safety Inspection",
            f"🚧 {location} Arena Closure Forces Event Relocations",
            f"⚠️ Multiple {location} Venues Close Due to Weather Conditions"
        ]
    }
    
    # General advisory news
    general_headlines = [
        f"📰 {location} Weekend Update: What You Need to Know Before Heading Out",
        f"🗓️ {location} Events Calendar: Changes and Updates for Today",
        f"📍 Around {location}: Local Happenings and Community News"
    ]
    
    # Build news based on weather condition
    if weather_condition:
        # Normalize weather condition
        condition_key = weather_condition
        for key in weather_headlines.keys():
            if key.lower() in weather_condition.lower() or weather_condition.lower() in key.lower():
                condition_key = key
                break
        
        if condition_key in weather_headlines:
            headlines = weather_headlines[condition_key]
            for i, headline in enumerate(headlines):
                article = {
                    "id": f"NEWS-WX-{random.randint(1000, 9999)}",
                    "category": "weather",
                    "headline": headline,
                    "summary": _generate_weather_summary(location, condition_key),
                    "source": random.choice(["Local News 5", "Weather Channel", "City Herald", "Metro Times"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "high" if "EMERGENCY" in headline or "BREAKING" in headline else "normal"
                }
                news_articles.append(article)
    
    # Build news based on context (transportation issues, sold out events, etc.)
    if context:
        context_lower = context.lower()
        
        # Check for transportation context
        if any(word in context_lower for word in ["flight", "airport", "plane"]):
            for headline in transport_headlines.get("flight_delays", [])[:2]:
                article = {
                    "id": f"NEWS-TR-{random.randint(1000, 9999)}",
                    "category": "transportation",
                    "headline": headline,
                    "summary": f"Transportation services in {location} are experiencing significant disruptions. Travelers are advised to check schedules and allow extra time for their journeys.",
                    "source": random.choice(["Traffic Report", "Transit Authority", "Travel Advisory"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "high"
                }
                news_articles.append(article)
        
        if any(word in context_lower for word in ["taxi", "uber", "lyft", "rideshare", "car"]):
            for headline in transport_headlines.get("taxi_shortage", [])[:2]:
                article = {
                    "id": f"NEWS-TX-{random.randint(1000, 9999)}",
                    "category": "transportation",
                    "headline": headline,
                    "summary": f"Rideshare and taxi availability in {location} is extremely limited. Consider public transit or postponing non-essential trips.",
                    "source": random.choice(["City Transport Desk", "Rideshare Monitor", "Local News"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "normal"
                }
                news_articles.append(article)
        
        if any(word in context_lower for word in ["traffic", "congestion", "road"]):
            for headline in transport_headlines.get("traffic", [])[:2]:
                article = {
                    "id": f"NEWS-TF-{random.randint(1000, 9999)}",
                    "category": "transportation",
                    "headline": headline,
                    "summary": f"Heavy traffic conditions are affecting {location}. Use navigation apps for real-time updates and consider alternative routes.",
                    "source": random.choice(["Traffic Watch", "City DOT", "Commuter Alert"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "normal"
                }
                news_articles.append(article)
        
        if any(word in context_lower for word in ["sold out", "unavailable", "no tickets"]):
            for headline in entertainment_headlines.get("sold_out", [])[:2]:
                article = {
                    "id": f"NEWS-EN-{random.randint(1000, 9999)}",
                    "category": "entertainment",
                    "headline": headline,
                    "summary": f"High demand has led to sold-out conditions for events in {location}. Check secondary markets or wait for additional dates.",
                    "source": random.choice(["Entertainment Weekly", "Event Insider", "Ticket Watch"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "normal"
                }
                news_articles.append(article)
        
        if any(word in context_lower for word in ["cancelled", "postponed", "closed"]):
            for headline in entertainment_headlines.get("cancelled", [])[:2]:
                article = {
                    "id": f"NEWS-CX-{random.randint(1000, 9999)}",
                    "category": "entertainment",
                    "headline": headline,
                    "summary": f"Event cancellations in {location} are affecting scheduled activities. Contact venues for refund information.",
                    "source": random.choice(["Event Update", "Venue News", "City Calendar"]),
                    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                    "priority": "high"
                }
                news_articles.append(article)
    
    # If no specific news, add general headlines
    if not news_articles:
        for headline in general_headlines[:2]:
            article = {
                "id": f"NEWS-GN-{random.randint(1000, 9999)}",
                "category": "general",
                "headline": headline,
                "summary": f"Stay informed about local happenings in {location}. Check back for updates on events, weather, and more.",
                "source": random.choice(["Local News", "City Pulse", "Community Update"]),
                "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                "priority": "low"
            }
            news_articles.append(article)
    
    result = {
        "status": "success",
        "location": location,
        "category": category,
        "articles_count": len(news_articles),
        "news": news_articles,
        "advisory": _generate_advisory(weather_condition, context) if (weather_condition or context) else None
    }
    
    return json.dumps(result)


def _generate_weather_summary(location: str, condition: str) -> str:
    """Generate a weather-specific news summary."""
    summaries = {
        "Storm": f"A powerful storm system is affecting the {location} area. Residents are advised to stay indoors, avoid unnecessary travel, and monitor local emergency channels for updates. Power outages have been reported in some areas.",
        "Severe Storm": f"EMERGENCY CONDITIONS: {location} is experiencing severe storm conditions with dangerous winds and heavy precipitation. All outdoor activities should be cancelled. Stay away from windows and seek shelter in interior rooms. Do not attempt to travel.",
        "Heavy Rain": f"Heavy rainfall is causing flooding in parts of {location}. Low-lying areas are particularly affected. Drivers should avoid flooded roads - remember, 'turn around, don't drown.' Flash flood warnings remain in effect.",
        "Snow": f"Significant snowfall is blanketing {location}, creating hazardous driving conditions. Road crews are working to clear main arteries, but side streets may remain impassable. Allow extra time for travel or postpone non-essential trips.",
        "Ice Storm": f"Ice accumulation in {location} has created extremely dangerous conditions on roads, sidewalks, and bridges. Multiple accidents have been reported. Stay off roads unless absolutely necessary. Watch for falling tree branches.",
        "Tornado Watch": f"Atmospheric conditions are favorable for tornado development in the {location} area. Residents should identify their safe shelter location and be prepared to take cover immediately if a tornado warning is issued.",
        "Hurricane Watch": f"A hurricane is approaching {location}. Residents in evacuation zones should follow official guidance. Secure outdoor furniture, stock up on supplies, and have an evacuation plan ready.",
        "Thunderstorm": f"Thunderstorm activity is moving through {location}. Lightning strikes have been reported. Seek shelter indoors away from windows. Avoid using corded phones and stay away from plumbing fixtures during the storm.",
        "Heat Advisory": f"Dangerously high temperatures are expected in {location}. Stay hydrated, limit outdoor activities during peak heat hours (10 AM - 6 PM), and check on elderly neighbors. Never leave children or pets in parked vehicles."
    }
    return summaries.get(condition, f"Weather conditions in {location} may affect your plans. Please check the latest forecast before heading out.")


def _generate_advisory(weather_condition: str = None, context: str = None) -> dict:
    """Generate an advisory message based on conditions."""
    if weather_condition and any(severe in weather_condition for severe in ["Severe", "Storm", "Tornado", "Hurricane"]):
        return {
            "level": "critical",
            "message": "🚨 TRAVEL NOT RECOMMENDED. Current conditions make outdoor activities dangerous. Consider postponing your plans or finding indoor alternatives.",
            "action": "Stay indoors and monitor local news for updates."
        }
    elif weather_condition and any(moderate in weather_condition for moderate in ["Rain", "Snow", "Thunder"]):
        return {
            "level": "warning",
            "message": "⚠️ TRAVEL WITH CAUTION. Weather conditions may cause delays and hazardous situations. Plan for extra travel time.",
            "action": "Allow extra time, dress appropriately, and have backup plans ready."
        }
    elif context and any(issue in context.lower() for issue in ["sold out", "cancelled", "unavailable"]):
        return {
            "level": "info",
            "message": "ℹ️ AVAILABILITY ISSUES. Some events or services may not be available as planned.",
            "action": "Consider alternatives and have backup options ready."
        }
    else:
        return {
            "level": "info",
            "message": "ℹ️ Stay informed about local conditions that may affect your plans.",
            "action": "Check for updates before heading out."
        }
    


## 🤖 LlmAgents with Mixture of Experts (MoE)

In [ ]:
gemini_model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

# ═══════════════════════════════════════════════════════════════════════════════
# ☀️ WEATHER FETCHER - Runs FIRST to establish canonical weather in session
# ═══════════════════════════════════════════════════════════════════════════════
# This agent fetches weather data and stores it in session.state["weather_data"]
# via output_key. All other agents can then read {weather_data} for consistency.

weather_fetcher = LlmAgent(
    name="weather_fetcher",
    model=gemini_model,
    instruction="""You are a Weather Data Fetcher. Your ONLY job is to get the weather forecast.

Call get_weather_forecast for the location mentioned in the user's request.
Use timeframe="now" for current conditions, or use "6h", "12h", "24h" for forecasts.
Set check_severe=True if there's concern about severe weather.

Return the complete weather data. This data will be stored in the session and 
used by ALL other agents to ensure consistent weather information across the plan.

IMPORTANT: Just fetch and return the weather data. Do not make recommendations.""",
    tools=[get_weather_forecast],
    output_key="weather_data"  # ← Stores weather in session for all agents to read!
)

# ☀️ EXPERT 1: Weather Expert (reads from session, provides analysis)
weather_expert = LlmAgent(
    name="weather_expert",
    model=gemini_model,
    instruction="""You are a Meteorology & Weather Planning Expert.

IMPORTANT: Weather data is available in the session: {weather_data}
Use this data for your analysis - do NOT call get_weather_forecast again.

Your expertise:
- Analyze the weather data from {weather_data}
- Identify weather risks and recommend precautions
- Advise on activity timing based on weather conditions
- Provide safety recommendations for outdoor activities

If {weather_data} shows concerning conditions, provide detailed warnings.
Focus on weather analysis - outfit recommendations are handled by Merchandising Expert.""",
    tools=[],  # No tools needed - reads from session
    output_key="weather_analysis"
)

# 🚗 EXPERT 2: Transportation Expert
# Reads weather from session for consistent weather impact analysis
transportation_expert = LlmAgent(
    name="transportation_expert",
    model=gemini_model,
    instruction="""You are a Transportation & Routing Expert.

IMPORTANT: Weather data is available in the session: {weather_data}
Extract the "condition" field from {weather_data} for weather impact analysis.

Your expertise:
- Recommend the BEST transport mode given constraints (time, cost, weather, availability)
- Provide alternative routes when primary options fail
- Analyze multi-leg journeys for optimal transfers
- Account for surge pricing, delays, and real-time conditions

Available tools:
- get_real_time_traffic(origin, destination, time): Traffic analysis with route alternatives
- book_taxi(destination, pickup_time, pickup_location, check_only): Unified taxi tool
  * check_only=True to just check availability/surge pricing without booking
  * check_only=False (default) to actually book the taxi
- get_transit_info(route, departure_time): Unified transit tool with platform, delays, crowding
- analyze_weather_impact(route, mode, weather_condition): Weather impact on transport modes
  * IMPORTANT: Pass the "condition" from {weather_data} as weather_condition parameter
  * Example: If {weather_data} shows "condition": "Rain", call analyze_weather_impact(route, "taxi", "Rain")

Always provide at least 2 alternatives and explain trade-offs.
Use the SAME weather condition from {weather_data} for all weather impact analyses.""",
    tools=[get_real_time_traffic, book_taxi, get_transit_info, analyze_weather_impact],
    output_key="transportation_analysis"
)

# 🛒 EXPERT 3: Merchandising Expert
# Reads weather from session for consistent outfit suggestions
merchandising_expert = LlmAgent(
    name="merchandising_expert",
    model=gemini_model,
    instruction="""You are a Merchandising & Shopping Expert with expertise in:
- Product search and inventory management
- Price comparison and deal hunting
- Promotion and discount optimization
- Personalized purchase recommendations
- Outfit recommendations based on weather (for shopping context)

IMPORTANT: Weather data is available in the session: {weather_data}
For outfit recommendations, extract temperature, condition, and precipitation from {weather_data}.

Your expertise:
- Find the best products matching user requirements
- Compare prices across vendors to find best deals
- Identify active promotions and discount opportunities
- Recommend items based on context, occasion, and budget
- Check inventory availability and alternatives
- Suggest appropriate outfits based on weather

Available tools:
- search_products(query, category, check_specific_item): Unified product search
  * check_specific_item="item name" to check availability of specific item
- find_best_deal(item, vendors, include_promotions): Unified deal finder
  * include_promotions=True (default) includes promo codes and discounts
- get_purchase_recommendations(context, budget): Personalized suggestions
- purchase_tickets(event, time, quantity): Buy event tickets
- get_outfit_recommendation(activity, temperature, condition, precipitation): Outfit based on weather
  * IMPORTANT: Pass values from {weather_data}:
    - temperature: from {weather_data}.temperature
    - condition: from {weather_data}.condition
    - precipitation: from {weather_data}.precipitation
  * Example: If {weather_data} shows temp=75, condition="Sunny", precipitation="None"
    call get_outfit_recommendation("dinner date", 75, "Sunny", "None")

Always prioritize value, quality, and availability in recommendations.
Use the SAME weather data from {weather_data} for outfit recommendations.""",
    tools=[search_products, find_best_deal, get_purchase_recommendations, purchase_tickets, get_outfit_recommendation],
    output_key="merchandising_analysis"
)

# 🛣️ PARALLEL EXPERT TEAM: MoE using ParallelAgent pattern
# All experts run CONCURRENTLY, each storing results in their output_key
# This is faster and follows ADK best practices for independent expert tasks
expert_team = ParallelAgent(
    name="expert_team",
    sub_agents=[weather_expert, transportation_expert, merchandising_expert],
)

# 📋 AGGREGATOR: Synthesizes parallel expert outputs into a unified plan
# Reads from all expert output_keys: {weather_analysis}, {transportation_analysis}, {merchandising_analysis}
planner_agent = LlmAgent(
    name="planner_agent",
    model=gemini_model,
    instruction="""You are an Expert Event Planner who synthesizes expert analyses into actionable plans.

EXPERT ANALYSES AVAILABLE (from parallel expert consultation):
- Weather Analysis: {weather_analysis}
- Transportation Analysis: {transportation_analysis}  
- Merchandising Analysis: {merchandising_analysis}

Weather context: {weather_data}

Your job is to SYNTHESIZE these expert analyses into a comprehensive, actionable plan:
1. Review all expert recommendations
2. Identify any conflicts or dependencies between recommendations
3. Create a unified timeline with specific action items
4. Ensure all logistics are covered (timing, transportation, purchases, weather considerations)
5. Highlight any warnings or risks mentioned by experts

Output a detailed, executable plan that incorporates insights from all experts.
The weather data consistency is guaranteed - all experts analyzed the same weather.""",
    tools=[],  # No AgentTools - experts already ran in parallel
    output_key="initial_plan"
)

# ⚠️ Risk Analyst Agent
# Has access to weather data for weather-related risk assessment
risk_analyst_agent = LlmAgent(
    name="risk_analyst",
    model=gemini_model,
    instruction="""You are a Risk Analyst. Review the plan from the planner: {initial_plan}

Weather context available: {weather_data}
Use this to assess weather-related risks accurately.

Identify potential failures in this plan:
- Weather disruptions (based on {weather_data} - storms, extreme temperatures)
- Transportation issues (traffic, taxi unavailability, transit delays)
- Sellouts and inventory problems (tickets, products out of stock)
- Timing conflicts and scheduling issues

For each identified risk, provide:
- Risk level (low/medium/high)
- Probability (estimated percentage)
- Potential impact on the plan
- Mitigation strategies

Be thorough and consider real-world failure scenarios.
Use the actual weather from {weather_data} for accurate weather risk assessment.""",
    output_key="risk_assessment"
)

# 🚪 Exit function for LoopAgent
def exit_defensive_loop():
    """Call this function when plan is validated and ready for execution."""
    return {"status": "approved", "message": "Plan validated - exiting defensive loop"}

# 🔧 Recovery Agent (uses consolidated tools + news for context)
recovery_agent = LlmAgent(
    name="recovery_agent",
    model=gemini_model,
    instruction="""You are a Recovery Agent responsible for fixing failed plans or approving successful ones.

Review the situation:
- Original plan: {initial_plan}
- Risk assessment: {risk_assessment}
- Weather context: {weather_data}

Your job:
1. Check if the plan has any tool failures (look for "status": "failure" in the plan text)

2. If FAILURES are found (plan cannot succeed):
   a) Call get_event_news to explain WHY the plan failed with relevant news headlines
   b) DO NOT call outfit coordination - user cannot attend the event
   c) Suggest alternatives or recommend staying home based on news

3. If NO failures detected AND risks are manageable (plan can succeed):
   a) Check if the plan mentions an OUTFIT or CLOTHING for the event
   b) If outfit is mentioned, call get_coordinated_outfit for A2A coordination:
      - This checks if other users attending the SAME event already claimed that color
      - If conflict detected, it suggests an alternative color
      - Pass: event_id (event name), activity, temperature/condition/precipitation from {weather_data}
      - preferred_color: extract the color from the plan (e.g., "red" from "red dress")
   c) After outfit coordination (if needed), call exit_defensive_loop to approve the plan

IMPORTANT - A2A OUTFIT COORDINATION:
When the plan SUCCEEDS and includes outfit details:
- Call get_coordinated_outfit(event_id, activity, temperature, condition, precipitation, preferred_color)
- The tool checks session state for colors already taken at that event
- If someone else (e.g., Alice) already claimed "red", it will suggest an alternative (e.g., "navy blue")
- This prevents fashion clashes when multiple users attend the same event

NEWS CONTEXT (only for failures):
- Severe storm → get_event_news(location, "weather", "Severe Storm") → "Stay home and watch TV"
- Taxi unavailable → get_event_news(location, "transportation", context="taxi unavailable")
- Tickets sold out → get_event_news(location, "entertainment", context="sold out")

Output format:
- For SUCCESS: Confirm plan approval, include A2A outfit coordination result if applicable
- For FAILURE: Show news context, explain why plan cannot proceed, suggest alternatives""",
    tools=[exit_defensive_loop, get_event_news, get_coordinated_outfit],
    output_key="initial_plan"  # Overwrites plan for next iteration
)



## 🔄 Defensive Loop Orchestrator

In [ ]:
# Build the Defensive Planning System using ADK workflow patterns
# WITH SESSION-BASED DATA CONSISTENCY

# Step 0: Weather Fetcher runs FIRST to establish canonical weather
# This stores weather in session.state["weather_data"] for all subsequent agents

# Step 1: Sequential workflow for Weather → Experts (parallel) → Planner → Risk
# Weather fetcher runs first, then experts run in PARALLEL, then planner aggregates, then risk analyst
planning_workflow = SequentialAgent(
    name="planning_workflow",
    sub_agents=[weather_fetcher, expert_team, planner_agent, risk_analyst_agent]
    # Data flow (MoE pattern with ParallelAgent):
    # 1. weather_fetcher → output_key="weather_data" → session.state
    # 2. expert_team (ParallelAgent) runs all experts CONCURRENTLY:
    #    - weather_expert → output_key="weather_analysis"
    #    - transportation_expert → output_key="transportation_analysis"
    #    - merchandising_expert → output_key="merchandising_analysis"
    # 3. planner_agent reads all expert outputs and synthesizes unified plan
    # 4. risk_analyst reviews plan with weather context
)

# Step 2: Loop workflow for defensive iterations (Workflow → Recovery → Retry)
# The loop continues until recovery agent calls exit_defensive_loop or max_iterations reached
defensive_loop = LoopAgent(
    name="defensive_loop",
    sub_agents=[planning_workflow, recovery_agent],
    max_iterations=2  # Allows up to 2 recovery attempts
)


class DefensiveOrchestrator:
    """
    Orchestrator using ADK workflow patterns.
    No manual threading or loop logic - delegates to ADK's LoopAgent and SequentialAgent.
    
    Supports optional session_service for A2A coordination scenarios.
    When session_service is provided, outfit coordination can happen across users.
    """
    def __init__(self, root_agent, session_service=None):
        self.root_agent = root_agent
        
        # If no session service provided, create a local in-memory one
        if session_service is None:
            self.session_service = InMemorySessionService()
        else:
            self.session_service = session_service
            
        # Always use standard Runner with the session service
        self.runner = Runner(
            agent=root_agent,
            app_name="defensive_planner",
            session_service=self.session_service
        )
    
    def execute_loop(self, goal, user_id):
        """
        Execute defensive planning using ADK workflow agents.
        The LoopAgent handles all iteration logic automatically.
        
        Args:
            goal: The user's planning goal
            user_id: User identifier for session tracking
        
        Returns:
            Tuple of (result_text, success_flag)
        """
        # 📊 OBSERVABILITY: Log session start and record metrics
        logger.info(f"🚀 SESSION START: user_id={user_id}, goal='{goal[:100]}{'...' if len(goal) > 100 else ''}'")
        metrics_collector.record_session_start()
        
        try:
            # Run the defensive loop - ADK handles all the workflow logic
            result = self.runner.run(self.root_agent, user_message=goal, user_id=user_id)
            
            # Extract final output
            final_output = result.output_message.content[0].text if result.output_message and result.output_message.content else None
            
            # Determine success/failure
            plan_succeeded = final_output and "failure" not in final_output.lower()
            
            # 📊 OBSERVABILITY: Log session completion
            logger.info(f"{'🏆' if plan_succeeded else '📰'} SESSION COMPLETE: user_id={user_id}, success={plan_succeeded}")
            metrics_collector.record_session_completion(plan_succeeded)
            
            return final_output, plan_succeeded
            
        except Exception as e:
            # 📊 OBSERVABILITY: Log session failure
            logger.error(f"💥 SESSION FAILED: user_id={user_id}, error={str(e)}")
            metrics_collector.record_session_completion(False)
            return None, False


## 🧪 Evaluation & Demonstration

This section performs **Scenario-Based Evaluation** to validate the agent's capabilities. We test the agent against 5 distinct scenarios to evaluate:

*   **Functional Correctness:** Can it create a valid plan?
*   **Resilience:** Can it recover from injected failures (Chaos Engineering)?
*   **Coordination:** Can it correctly resolve multi-user conflicts (A2A)?

**Scenarios:**
1.  **Standard Planning:** Simple requests (Scenario 1).
2.  **Contextual Reasoning:** Weather + Transport (Scenario 2).
3.  **Complex Orchestration:** Multi-step plans with MoE (Scenario 3).
4.  **Resilience (Chaos Engineering):** Recovery from tool failures (Scenario 4).
5.  **A2A Coordination:** Multi-user conflict resolution (Scenario 5).

**Expected behavior:**
- Planner agent will use `AgentTool` to call transportation expert
- No manual string parsing - LLM naturally calls the tool
- Expert runs and returns analysis via `output_key`
- Planner synthesizes recommendation into plan

In [ ]:
# Initialize orchestrator
orchestrator = DefensiveOrchestrator(root_agent=defensive_loop)

def run_scenario(name: str, goal: str, user_id: str, orchestrator_instance=None):
    """
    Evaluation wrapper that displays scenario results.
    All display logic is here, keeping orchestrator clean.
    """
    orch = orchestrator_instance or orchestrator
    
    print("\n" + "#" * 80)
    print(f"# {name}")
    print("#" * 80)
    print(f"🎯 USER: {user_id}")
    print(f"🎯 GOAL: {goal[:100]}{'...' if len(goal) > 100 else ''}")
    print("═" * 80)
    
    result, success = orch.execute_loop(goal=goal, user_id=user_id)
    
    print("\n" + "═" * 80)
    if success:
        print("🏆 DEFENSIVE PLANNING COMPLETE")
    else:
        print("📰 PLANNING COMPLETE (with news/fallback)")
    print("═" * 80)
    print(f"\n{result}\n")
    print("═" * 80)
    print(f"\n📊 OUTCOME: {'SUCCESS ✅' if success else 'FAILED ❌'}")
    
    return result, success


### Scenario 1: Art Exhibition

In [ ]:
result1, success1 = run_scenario(
    name="SCENARIO 1: Transportation Expert",
    goal="Attend Modernist Gala exhibition at 7 PM downtown - need optimal route",
    user_id="Scenario1_User"
)


### Scenario 2: Flight Connection

In [ ]:
result2, success2 = run_scenario(
    name="SCENARIO 2: Weather + Transportation",
    goal="Outdoor concert at Central Park tomorrow 6 PM - what should I wear and how to get there?",
    user_id="Scenario2_User"
)


### Scenario 3: Date Night (Sold Out Restaurant)

In [ ]:
result3, success3 = run_scenario(
    name="SCENARIO 3: Complete MoE",
    goal="""Plan anniversary dinner date:
    - Dinner reservation at The Riverside Restaurant downtown at 7:30 PM
    - Need to buy a nice gift first (jewelry or perfume, budget $150)
    - What should I wear given the weather?
    - Best route from my office in midtown
    """,
    user_id="Scenario3_User"
)


### Scenario 4: Concert (Transportation Breakdown)

In [ ]:
result4, success4 = run_scenario(
    name="SCENARIO 4: Defensive Recovery",
    goal="""Complex holiday shopping trip:
    - Buy gifts at mall (3 items, budget $300)
    - Need weather forecast for afternoon outdoor market visit
    - Traffic from suburbs to downtown during rush hour
    - Purchase theater tickets for evening show
    """,
    user_id="Scenario4_User"
)

if not success4:
    print("\n⚠️  Plan exceeded max recovery attempts - demonstrates:")
    print("  • LoopAgent max_iterations enforcement")
    print("  • Chaos engineering causing repeated failures")
    print("  • Recovery agent attempting alternatives")
    print("  • Graceful failure handling")
else:
    print("\n✅ LoopAgent successfully recovered from chaos-injected failures!")


### Scenario 5: Agent-to-Agent (A2A) Social Conflict

In [ ]:
print("\n" + "#" * 80)
print("# SCENARIO 5: Agent-to-Agent (A2A) Social Conflict Resolution")
print("#" * 80)

# Create shared session service for A2A coordination
a2a_session_service = InMemorySessionService()
A2A_EVENT_ID = "sarahs_wedding_grand_ballroom"

# Use DefensiveOrchestrator with session_service
a2a_orchestrator = DefensiveOrchestrator(
    root_agent=defensive_loop,
    session_service=a2a_session_service
)

print("═" * 80)
print("🔗 A2A SETUP")
print("═" * 80)
print(f"   Event: {A2A_EVENT_ID}")
print("   Orchestrator: DefensiveOrchestrator with session_service")
print("   Coordination: get_coordinated_outfit tool (in recovery_agent)")
print("   State: ToolContext.state[event:{id}:outfits]")
print()

# ALICE: Plans to wear Red Dress
print("═" * 80)
print("👩 ALICE: Planning for wedding - wants RED DRESS")
print("═" * 80)

alice_result, alice_success = a2a_orchestrator.execute_loop(
    user_id="Alice",
    goal=f"""Attend Sarah's wedding at Grand Ballroom at 4 PM.
    I want to wear my RED DRESS. Plan transportation and coordinate my outfit.
    Event ID for outfit coordination: {A2A_EVENT_ID}
    My name is Alice and my preferred color is red."""
)

if alice_success:
    print(f"\n{alice_result}\n")

# BOB: Also wants Red - A2A should detect conflict
print("\n" + "═" * 80)
print("👨 BOB: Planning for SAME wedding - wants RED SUIT")
print("═" * 80)
print("   (get_coordinated_outfit should detect Alice already has red)")

bob_result, bob_success = a2a_orchestrator.execute_loop(
    user_id="Bob",
    goal=f"""Attend Sarah's wedding at Grand Ballroom at 4 PM.
    I want to wear my RED SUIT. Plan transportation and coordinate my outfit.
    Event ID for outfit coordination: {A2A_EVENT_ID}
    My name is Bob and my preferred color is red."""
)

if bob_success:
    print(f"\n{bob_result}\n")

# A2A RESULTS SUMMARY
print("\n" + "═" * 80)
print("🔗 A2A COORDINATION RESULTS")
print("═" * 80)

a2a_success = alice_success and bob_success

# Check if Bob got an alternative color
conflict_resolved = False
bob_color = "red"
if bob_result:
    bob_lower = bob_result.lower()
    for alt in ["navy blue", "emerald green", "burgundy", "charcoal", "forest green", "royal purple"]:
        if alt in bob_lower:
            bob_color = alt
            conflict_resolved = True
            break
    if not conflict_resolved and ("conflict" in bob_lower or "alternative" in bob_lower or "changed" in bob_lower):
        conflict_resolved = True
        bob_color = "alternative color"

print(f"\n👩 Alice: {'SUCCESS' if alice_success else 'FAILED'}")
if alice_success:
    print("   → Red dress registered in session state")
else:
    print("   → Could not attend (news provided)")

print(f"\n👨 Bob: {'SUCCESS' if bob_success else 'FAILED'}")
if bob_success:
    if conflict_resolved:
        print(f"   → Wanted red, got {bob_color} (conflict resolved!)")
    else:
        print("   → Red suit (no conflict detected or same color allowed)")
else:
    print("   → Could not attend (news provided)")

print("\n" + "═" * 80)
print("🎯 A2A SCENARIO OUTCOME:")
if a2a_success and conflict_resolved:
    print("   ✅ SUCCESS: get_coordinated_outfit prevented fashion conflict!")
    print(f"   ✅ Bob's color changed from red to {bob_color}")
elif a2a_success:
    print("   ⚠️  Both succeeded - check if recovery_agent called get_coordinated_outfit")
elif alice_success and not bob_success:
    print("   📰 Alice succeeded, Bob failed (news shown, no outfit coordination)")
elif not alice_success and bob_success:
    print("   📰 Alice failed, Bob succeeded")
else:
    print("   📰 Both failed - chaos engineering in action!")
print("═" * 80)

# Show metrics at the end
print(f"\n{metrics_collector.report()}")


## 📊 Evaluation Methodology Note

For this Capstone, **Agent Evaluation** is demonstrated through the functional scenarios above. 

In a production environment, we would extend this using the **Google ADK Evaluation Framework** to run:
1.  **Batch Evaluation:** Running the agent against a dataset of 50+ planning queries.
2.  **Metric Collection:** Measuring "Success Rate" (plans completed vs. failed), "Recovery Rate" (failures fixed / total failures), and "Latency".
3.  **Human Review:** Using a "Golden Dataset" of ideal plans to compare against agent outputs.

The scenarios above serve as a representative sample of this evaluation process, proving the agent's core competencies in planning, recovery, and coordination.